<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/extrai_info_audio_e_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install eyeD3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 22.1 MB/s eta 0:00:00


In [6]:
!apt-get install ffmpeg libavcodec-extra > /dev/null

In [13]:
!pip3 install ffmpeg-python

In [19]:
import os
import datetime
import subprocess
import json
import ffmpeg

def extract_audio_info(file_path):
  """
  Extracts audio information using FFmpeg.

  Args:
      file_path (str): Path to the audio file.

  Returns:
      dict: Dictionary containing extracted information.
  """
  info = {}
  command = ["ffprobe", "-v", "quiet", "-print_format", "json", "-show_format", "-show_streams", file_path]
  try:
    output = subprocess.check_output(command, encoding="utf-8")
    data = json.loads(output)
    format = data["format"]
    info["Nome do arquivo"] = os.path.basename(file_path)
    info["Tamanho do arquivo"] = os.path.getsize(file_path)
    info["Data de upload"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    info["Tipo MIME"] = format.get("format_name")

    # Extract duration from streams
    for stream in data["streams"]:
      if stream["codec_type"] == "audio":
        duration = stream.get("duration", "N/A")
        info["Duração"] = f"{float(duration):.2f} segundos"
        # Extract additional audio information
        info["Taxa de bits"] = f"{stream.get('bit_rate', 'N/A')} kbps"
        info["Taxa de amostragem"] = f"{stream.get('sample_rate', 'N/A')} Hz"
        info["Canais"] = stream.get("channels", 'N/A')
        break

  except (subprocess.CalledProcessError, json.JSONDecodeError) as e:
    print(f"Erro ao extrair informações: {e}")

  return info

def extract_video_info(file_path):
  """
  Extrai informações de um arquivo de vídeo usando FFmpeg.

  Args:
      file_path (str): Caminho para o arquivo de vídeo.

  Returns:
      dict: Dicionário com as informações do arquivo de vídeo.
  """
  info = {}
  probe = ffmpeg.probe(file_path)

  # Extrair informações básicas
  info["Nome do arquivo"] = os.path.basename(file_path)
  info["Tamanho do arquivo"] = os.path.getsize(file_path)
  info["Data de upload"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  info["Tipo MIME"] = probe.get("format").get("mime_type")

  # Extract video stream
  video_stream = next((stream for stream in probe["streams"] if stream["codec_type"] == "video"), None)

  # Extract video information (if stream is found)
  if video_stream is not None:
    # Convert duration to float before formatting
    duration_in_seconds = float(video_stream['duration'])
    info["Duração"] = f"{duration_in_seconds:.2f} segundos"

    info["Resolução"] = f"{video_stream['width']}x{video_stream['height']}"
    info["Taxa de quadros"] = f"{video_stream['r_frame_rate']} fps"
    info["Codec de vídeo"] = video_stream["codec_name"]

  # Extrair informações de áudio
  audio_stream = next((stream for stream in probe["streams"] if stream["codec_type"] == "video"), None)
  if audio_stream is not None:
    info["Codec de áudio"] = audio_stream.get("codec_name")

  # Extrair taxa de bits
  info["Taxa de bits"] = f"{probe.get('bit_rate')} kbps"

  return info

In [20]:
# Caminho para o arquivo de áudio
file_audio_path = "/content/Kalimba.mp3"

# Extrair informações do arquivo
audio_info = extract_audio_info(file_audio_path)

# Imprimir informações do arquivo
for key, value in audio_info.items():
  print(f"{key}: {value}")

Nome do arquivo: Kalimba.mp3
Tamanho do arquivo: 8414449
Data de upload: 2024-02-29 03:11:22
Tipo MIME: mp3
Duração: 348.06 segundos
Taxa de bits: 192000 kbps
Taxa de amostragem: 44100 Hz
Canais: 2


In [21]:
file_video_path = "/content/mycat.mp4"
video_info = extract_video_info(file_video_path)
for key, value in video_info.items():
  print(f"{key}: {value}")

Nome do arquivo: mycat.mp4
Tamanho do arquivo: 10652213
Data de upload: 2024-02-29 03:11:30
Tipo MIME: None
Duração: 54.87 segundos
Resolução: 480x848
Taxa de quadros: 30/1 fps
Codec de vídeo: h264
Codec de áudio: h264
Taxa de bits: None kbps
